In [290]:
import pandas as pd
import numpy as np
import pandas_profiling
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from datetime import date
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.utils import shuffle
from scipy.stats import randint as sp_randint
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [291]:
train = pd.read_csv("flights_train.csv",thousands=',')
train.head()

,UID,DAY_OF_WEEK,CRS_DEP_TIME,TAXI_OUT,TAXI_IN,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,AIRLINE_ID,FL_DATE,...,FL_NUM,UNIQUE_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_CITY_NAME,DEST_STATE_ABR,FIRST_DEP_TIME,ARR_DELAY
0,0,2,1020,11,8,373,2701,11,19930,1/3/17,...,879,AS,SEA,"Seattle, WA",WA,LIH,"Lihue, HI",HI,NaN,-11
1,1,6,1220,13,9,183,1171,5,19805,1/28/17,...,2491,AA,SAN,"San Diego, CA",CA,DFW,"Dallas/Fort Worth, TX",TX,NaN,1
2,2,6,945,48,6,355,2552,11,19930,1/21/17,...,172,AS,ANC,"Anchorage, AK",AK,PHX,"Phoenix, AZ",AZ,NaN,18
3,3,1,1437,17,8,136,967,4,19977,1/23/17,...,2030,UA,SFO,"San Francisco, CA",CA,DEN,"Denver, CO",CO,NaN,0
4,4,1,830,37,7,114,534,3,19393,1/2/17,...,3812,WN,ATL,"Atlanta, GA",GA,IAD,"Washington, DC",VA,NaN,197


In [292]:
pandas_profiling.ProfileReport(train)

Number of variables,22
Number of observations,4911
Total Missing (%),0.0%
Total size in memory,844.1 KiB
Average record size in memory,176.0 B
Numeric,11
Categorical,8
Boolean,0
Date,0
Text (Unique),0
Rejected,3


In [293]:
def oneHotEncoding(df,feature):
    dummies = pd.get_dummies(df[feature], prefix=feature)
    df = pd.concat([df,dummies],axis=1)
    df = df.drop([feature],axis=1)
    return df

In [294]:
train['FL_DATE'] = pd.to_datetime(train['FL_DATE'])
cal = calendar()
holidays = cal.holidays(start=train['FL_DATE'].min(), end=train['FL_DATE'].max())
train['IS_HOLIDAY'] = train['FL_DATE'].isin(holidays)
train['IS_HOLIDAY'] = np.where(train['IS_HOLIDAY'] == True,1,0)
train['SPEED'] = train['DISTANCE']/train['ACTUAL_ELAPSED_TIME']
train['Day'] = train['FL_DATE'].dt.day
train['Month'] = train['FL_DATE'].dt.month
train['IS_WEEKEND'] = train.apply(lambda _: '', axis=1)
weekends = [6,7]
train['IS_WEEKEND'] = np.where(train['DAY_OF_WEEK'].isin(weekends), 1, 0)
train = oneHotEncoding(train,'AIRLINE_ID')

In [295]:
# dropping columns with high categorical values and highly correlated columns

def dropHighCategoricalColumns(train):
    train = train.drop(columns=['ORIGIN_STATE_ABR','UNIQUE_CARRIER','DEST_STATE_ABR','UID','DEST','DEST_CITY_NAME','DISTANCE_GROUP','FIRST_DEP_TIME','FL_DATE','ORIGIN','ORIGIN_CITY_NAME','ACTUAL_ELAPSED_TIME','CRS_DEP_TIME','FL_NUM'])
    return train

In [296]:
train = dropHighCategoricalColumns(train)

In [297]:
test = pd.read_csv("flights_test.csv",thousands=',')
test_output = pd.DataFrame(data = test['UID'])
test['FL_DATE'] = pd.to_datetime(test['FL_DATE'])
holidays = cal.holidays(start=test['FL_DATE'].min(), end=test['FL_DATE'].max())
test['IS_HOLIDAY'] = test['FL_DATE'].isin(holidays)
test['IS_HOLIDAY'] = np.where(test['IS_HOLIDAY'] == True,1,0)
test['SPEED'] = test['DISTANCE']/test['ACTUAL_ELAPSED_TIME']
test['Day'] = test['FL_DATE'].dt.day
test['Month'] = test['FL_DATE'].dt.month
test['IS_WEEKEND'] = test.apply(lambda _: '', axis=1)
weekends = [6,7]
test['IS_WEEKEND'] = np.where(test['DAY_OF_WEEK'].isin(weekends), 1, 0)
test = oneHotEncoding(test,'AIRLINE_ID')
test = dropHighCategoricalColumns(test)

In [298]:
missing_cols = set( train.columns ) - set( test.columns )
for c in missing_cols:
    test[c] = 0
test = test[train.columns]

In [299]:
train.head()

,DAY_OF_WEEK,TAXI_OUT,TAXI_IN,DISTANCE,ORIGIN_CITY_MARKET_ID,DEST_CITY_MARKET_ID,ARR_DELAY,IS_HOLIDAY,SPEED,Day,...,AIRLINE_ID_19790,AIRLINE_ID_19805,AIRLINE_ID_19930,AIRLINE_ID_19977,AIRLINE_ID_20304,AIRLINE_ID_20366,AIRLINE_ID_20409,AIRLINE_ID_20416,AIRLINE_ID_20436,AIRLINE_ID_21171
0,2,11,8,2701,30559,32982,-11,0,7.241287,3,...,0,0,1,0,0,0,0,0,0,0
1,6,13,9,1171,33570,30194,1,0,6.398907,28,...,0,1,0,0,0,0,0,0,0,0
2,6,48,6,2552,30299,30466,18,0,7.188732,21,...,0,0,1,0,0,0,0,0,0,0
3,1,17,8,967,32457,30325,0,0,7.110294,23,...,0,0,0,1,0,0,0,0,0,0
4,1,37,7,534,30397,30852,197,1,4.684211,2,...,0,0,0,0,0,0,0,0,0,0


In [300]:
#train = shuffle(train)
Y = train['ARR_DELAY']
Y.values.reshape(train.shape[0],1)
train = train.drop(['ARR_DELAY'],axis = 1)

In [301]:
regr = linear_model.LinearRegression()
param_grid = {'fit_intercept': [True, False],
                      'normalize': [True, False]}
grid_search = GridSearchCV(regr, param_grid=param_grid,scoring='neg_mean_squared_error',cv = 10)
grid_search.fit(train, Y)

GridSearchCV(cv=10, error_score='raise',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'normalize': [True, False], 'fit_intercept': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [302]:
print -1*grid_search.best_score_ , grid_search.best_params_

1940.90185776 {'normalize': False, 'fit_intercept': True}


In [303]:
decision_tree = DecisionTreeRegressor()
param_grid = {
            "splitter" : ["best","random"],
            "max_features" : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8],
            "max_depth" : [2,4,6,8,10],
            "presort" : [True]
            }

grid_search = GridSearchCV(decision_tree, param_grid=param_grid,scoring='neg_mean_squared_error',cv = 10)
grid_search.fit(train, Y)

GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_features': ['auto', 'sqrt', 'log2'], 'splitter': ['best', 'random'], 'min_samples_split': [2, 4, 8], 'max_depth': [2, 4, 6, 8, 10], 'presort': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [304]:
print -1*grid_search.best_score_, grid_search.best_params_

2010.27929625 {'max_features': 'auto', 'min_samples_split': 2, 'presort': True, 'max_depth': 2, 'splitter': 'random'}


In [220]:
random_forest_regressor = RandomForestRegressor()
# specify parameters and distributions to sample from
# use a full grid over all parameters
param_grid = { 
            "n_estimators"      : [10,20,30,40,50,60],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8],
            "bootstrap": [True, False],
            "warm_start" : [True, False]
            }

# run grid search
grid_search = GridSearchCV(random_forest_regressor, param_grid=param_grid,scoring='neg_mean_squared_error',cv = 10)
grid_search.fit(train, Y)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 20, 30, 40, 50, 60], 'max_features': ['auto', 'sqrt', 'log2'], 'min_samples_split': [2, 4, 8], 'warm_start': [True, False], 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [221]:
print -1*grid_search.best_score_, grid_search.best_params_

1985.79949748 {'max_features': 'log2', 'min_samples_split': 8, 'bootstrap': True, 'n_estimators': 60, 'warm_start': True}


In [319]:
gradient_booster = GradientBoostingRegressor()
# specify parameters and distributions to sample from
# use a full grid over all parameters
param_grid = {
            "n_estimators"      : [40,50,60],
            "learning_rate"     : [0.05,0.1,0.2,0.4,0.8],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,6,8],
            "warm_start" : [True, False],
            "loss" : ["ls", "lad", "huber", "quantile"]
            }

# run grid search
grid_searchGB = GridSearchCV(gradient_booster, param_grid=param_grid,scoring='neg_mean_squared_error',cv = 10)
grid_searchGB.fit(train, Y)

GridSearchCV(cv=10, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'warm_start': [True, False], 'loss': ['ls', 'lad', 'huber', 'quantile'], 'learning_rate': [0.05, 0.1, 0.2, 0.4, 0.8], 'n_estimators': [40, 50, 60], 'min_samples_split': [2, 4, 6, 8], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [320]:
print -1*grid_searchGB.best_score_, grid_searchGB.best_params_

1945.63789283 {'warm_start': False, 'loss': 'huber', 'learning_rate': 0.8, 'n_estimators': 60, 'min_samples_split': 4, 'max_features': 'log2'}


## Predicting on Test Data

In [305]:
test = test.drop(columns = ['ARR_DELAY'])

In [328]:
y_pred = grid_searchGB.predict(test)

In [329]:
test_output['ARR_DELAY'] = y_pred

In [330]:
test_output

,UID,ARR_DELAY
0,8,-6.854685
1,15,-0.523658
2,29,-11.335012
3,33,-18.915505
4,65,-24.685054
5,79,11.807275
6,80,-0.582093
7,84,-3.409686
8,102,-0.089817
9,106,1.690401


In [334]:
test_output.to_csv("test_outputs.csv")